<a href="https://colab.research.google.com/github/danielbauer1979/MSDIA_PredictiveModelingAndMachineLearning/blob/main/GB888_IV_4_Bagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bagging

In this tutorial, we discuss approaches to improve on the predictive porformance of CARTs via *aggregation*. We consider a basic bootstrap aggregation approach in the context of a simple example with a single predictor, illustrating key aspects and pitfalls.

As usually, let's start with loading the relevant libaries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm
import seaborn as sns
from sklearn.tree import DecisionTreeRegressor

And let's define:

In [ ]:
def sigmoid(x):
    return(1 / (1 + np.exp(-x)))

## Improvement via Aggregation

### Review of Concepts and Maths

As we have discussed in GB 656, there is a key tradeoff between *bias* and *variance*:  A more complex learner may perform marvelously *in-sample*, but the *out-of-sample* performance is poor due to *overfitting*.  *Ensemble* learning techniques use the input from basic learners trained on different data sets (or also the input from different learners trained on the same data set -- this is referred to as *stacking*), to improve on the predictive performance of the basic learner.

In *Bagging* -- short for *bootstrap-aggregating* -- the idea is to sample from the original dataset to obtain $M$ bootstrap samples, which are in turn used for training the predictive model yielding predictions $\hat{Y}_j,$ $j=1,\ldots,M.$  The final prediction $\hat{Y}$ then is based on the average of the different predictions: $\hat{Y} = \frac{1}{M} \sum_{j=1}^M \hat{Y}_j$.  

For instance, if each of the $M$ predictions is *unbiased*, $\mathbb{E}_x[\hat{Y}_i] = Y,$ then of course the aggregated prediction will be unbiased as well: $\mathbb{E}_x[\hat{Y}] = Y.$ However, we generally have for the standard deviation:
$$
\text{StDev}_x(\hat{Y}) = \frac{1}{N} \text{StDev}_x\left(\sum_{j=1}^M \hat{Y}_j\right) \leq \frac{1}{N} \sum_{j=1}^M \text{StDev}_x(Y_j),
$$
where the inequality is sharp if the predictions are not perfectly positively correlated.  Hence, aggregating can reduce the variance!

### A Simulated Example with One Predictor

Let us revisit the simple example from the previous tutorial on trees.  As a reminder, there we used the so-called *sigmoid* function that can depict highly linear as well as highly non-linear relationships by different choices of its parameter.  We used different parameters to generate two data sets, and compared the predictive preformance of trees vs. that of OLS regression.  Our conclusion was that trees work well in the non-linear situation whereas (linear) regression works well in the linear situation:


1. Generate the datasets:

In [ ]:
np.random.seed(42)
x = 3 * np.random.normal(0, 1, 150)
eps = 0.25 * np.random.normal(0, 1, 150)
y_1 = sigmoid(0.5 * x) + eps
y_2 = sigmoid(5 * x) + eps
mydata1 = pd.DataFrame({'y_1':y_1,'x':x})
mytraindata1 = mydata1[0:100]
mytestdata1 = mydata1[100:150]
mydata2 = pd.DataFrame({'y_2':y_2,'x':x})
mytraindata2 = mydata2[0:100]
mytestdata2 = mydata2[100:150]


2. Fit an OLS regression to the first dataset:

In [ ]:
lmfit1 = smf.ols(formula="y_1 ~ x", data=mytraindata1).fit()
yhat_OOS1 = lmfit1.predict(mytestdata1)
OLS_OOS_MSE1 = sum((mytestdata1['y_1'] - yhat_OOS1)**2)/50
OLS_OOS_MSE1

3. Fit a tree:

In [ ]:
 tree1 = DecisionTreeRegressor(max_leaf_nodes=2)
X = mytraindata1['x'].values.reshape(-1, 1)
y = mytraindata1['y_1'].values
tree1.fit(X, y)
ytreehat1 = tree1.predict(mytestdata1['x'].values.reshape(-1, 1))
TREE_OOS_MSE1 = sum((mytestdata1['y_1'] - ytreehat1)**2)/50
TREE_OOS_MSE1

Now, rather than generating one tree, let's contemplate an alternative.  Let us draw new data sets from sampling from the original data set (*bootstrapping*), let's fit an (unpruned) tree to each of the sampled data sets, and let's predict by averaging over the predictions of these new trees.

In [ ]:
ybaggedtreehat1 = np.zeros(mytestdata1['y_1'].shape)
atree = DecisionTreeRegressor()
for i in range(0, 100):
  subset = np.random.choice(len(mytraindata1), 25, replace=True)
  X = mytraindata1['x'][subset].values.reshape(-1, 1)
  y = mytraindata1['y_1'][subset].values
  atree.fit(X, y)
  ybaggedtreehat1 = ybaggedtreehat1 + atree.predict(mytestdata1['x'].values.reshape(-1, 1))
ybaggedtreehat1 = ybaggedtreehat1/100
BAGGED_MSE1 = sum((mytestdata1['y_1'] - ybaggedtreehat1)**2)/50
BAGGED_MSE1

We notice that by aggregating, the tree-based predictions perform notably better than the single tree.  What is going on?  There are two aspects worth noting:

1. As explained above, by aggregating the individual trees we potentially reduce the variance of the prediction.

2. We are not pruning the individual trees which leads to lower bias.  While this may lead to overfitting by any individual tree, we control the variance by subsequently averaging.  Thus,  we potentially reduce the bias of the prediction.

Let's compare the predictions:

In [ ]:
plt.scatter(mytestdata1['x'], mytestdata1['y_1'], c = 'k')
plt.plot(mytestdata1['x'], yhat_OOS1, c = 'k')
plt.scatter(mytestdata1['x'], ytreehat1, c = 'r')
plt.scatter(mytestdata1['x'], ybaggedtreehat1, c = 'b')

Let's also try the second dataset:

1. Tree:

In [ ]:
tree2 = DecisionTreeRegressor(max_leaf_nodes=2)
X = mytraindata2['x'].values.reshape(-1, 1)
y = mytraindata2['y_2'].values
tree2.fit(X, y)
ytreehat2 = tree2.predict(mytestdata2['x'].values.reshape(-1, 1))
TREE_OOS_MSE2 = sum((mytestdata2['y_2'] - ytreehat2)**2)/50
TREE_OOS_MSE2

2. Bagging:

In [ ]:
ybaggedtreehat2 = np.zeros(mytestdata2['y_2'].shape)
atree = DecisionTreeRegressor()
for i in range(0, 100):
  subset = np.random.choice(len(mytraindata2), 30, replace=True)
  X = mytraindata2['x'][subset].values.reshape(-1, 1)
  y = mytraindata2['y_2'][subset].values
  atree.fit(X, y)
  ybaggedtreehat2 = ybaggedtreehat2 + atree.predict(mytestdata2['x'].values.reshape(-1, 1))
ybaggedtreehat2 = ybaggedtreehat2/100
BAGGED_MSE2 = sum((mytestdata2['y_2'] - ybaggedtreehat2)**2)/50
BAGGED_MSE2

So it turns out here the aggregated sample beats even the basic tree model, which performed quite well.  But why?  Let's look at the predictions:

In [ ]:
lmfit2 = smf.ols(formula="y_2 ~ x", data=mytraindata2).fit()
yhat_OOS2 = lmfit2.predict(mytestdata2)
plt.scatter(mytestdata2['x'], mytestdata2['y_2'], c = 'k')
plt.plot(mytestdata2['x'], yhat_OOS2, c = 'k')
plt.scatter(mytestdata2['x'], ytreehat2, c = 'r')
plt.scatter(mytestdata2['x'], ybaggedtreehat2, c = 'b')

So in the extremal areas, the predictions are similar, but the predictions of the aggregated estimator are smooth around the cutoff area.  A single tree that has one precise cutoff is likely to get it (slightly) wrong, so the smoothed transition (reflecting some ambiguity of whether the points in this area should be "up" or "down") generally performs better.

Now contemplate the (more realistic) situation where it is not ex ante clear of whether the relationship is more linear or more non-linear.  In this case, the aggregated estimator may be a conservative choice -- and it appears to definitely outperform the tree!